<a href="https://colab.research.google.com/github/RahulVerma2444/LangChain-Currency-Conversion-Agent/blob/main/mobileLive_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = ""


In [ ]:
%pip install --upgrade --quiet  langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
import requests
url = "https://api.frankfurter.dev/v1/latest?from={base}&to={target}"

response = requests.get(url)


print(response.json())

{'message': 'not found'}


In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)

In [ ]:
import requests
def convert_currency(amount, base_currency, target_currency):
    url = f"https://api.frankfurter.dev/v1/latest?amount={amount}&from={base_currency}&to={target_currency}"
    response = requests.get(url)
    return response.json()

In [ ]:
from langchain.tools import Tool
import re

def handle_currency_query(query: str) -> str:
    match = re.search(r"(\d+\.?\d*)\s*([A-Z]{3})\b.*?\b(to|in)\b.*?\b([A-Z]{3})", query.upper())

    if not match:
        return "Sorry, I couldn't extract the amount and currencies. Try something like 'Convert 100 USD to EUR'."

    amount, from_currency, _, to_currency = match.groups()

    # Call your real converter here (API or logic)
    text = convert_currency(amount, from_currency, to_currency)
    return text  # placeholder


In [ ]:
from langchain.tools import Tool
import re

def handle_currency_query(query: str):
    match = re.search(r"(?i)(\d+(?:\.\d+)?)\s*([A-Z]{3})\s*(?:to|in)\s*([A-Z]{3})", query)

    if not match:
        return "Sorry, I couldn't extract the amount and currencies. Try something like 'Convert 100 USD to EUR'."

    amount, from_currency, to_currency = match.groups()

    try:
        # Assuming convert_currency returns the converted amount directly
        converted_amount = convert_currency(amount, from_currency, to_currency)
        return converted_amount
    except Exception as e:
        return f"An error occurred during currency conversion: {e}"

In [ ]:
currency_tool = Tool(
    name="CurrencyConverter",
    func=handle_currency_query,
    description=(
        "Converts money from one currency to another"
        "Use when user asks to convert between currencies"
        "Input example: 'Convert 100 USD to EUR' or 'What is 250 CAD in GBP?'"
    )

)

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.agents.agent_types import AgentType


agent = initialize_agent(
    tools=[currency_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



In [ ]:
agent.run("Help me find what 293 USD is in EUR")



> Entering new AgentExecutor chain...
 I need to use the CurrencyConverter tool to convert USD to EUR
Action: CurrencyConverter
Action Input: Convert 293 USD to EUR
Observation: {'amount': 293.0, 'base': 'USD', 'date': '2025-07-18', 'rates': {'EUR': 251.51}}
Thought: I now know the final answer
Final Answer: 251.51 EUR

> Finished chain.


'251.51 EUR'